Leung Wai Liu <br>
JPMC <br>
July 25, 2022 <br>
Twitter Relabelling WHOLE SENTENCE

In [1]:
import pandas as pd
import numpy as np
import wordninja
import spacy
import re
from collections import Counter

In [2]:
nlp = spacy.load('en_core_web_trf')

In [3]:
combined_df = pd.read_csv('logs/combined_final.tsv', sep='\t')
combined_df

,Word,Word Index,POS Label,NER Label,Tweet
0,poutine,11,PROPN,O,rt @user1787 : i asked siri to remind me about...
1,friday,7,PROPN,O,rt @user912 : you know it 's friday when the p...
2,@user444,17,PROPN,O,mikel obi & his girl olga diyachenko spend hol...
3,@user1598,1,PROPN,O,""" @user1598 : super junior ’s siwon transforms..."
4,gospel,7,PROPN,O,""" do n't move on from the gospel , move into t..."
...,...,...,...,...,...
1013,@user70,3,PROPN,O,rt @user2041 : @user70 happy birthday man ! le...
1014,@user660,5,PROPN,O,rt @user1157 : happy birthday @user660 ! url1209
1015,@user2046,0,PROPN,O,@user2046 further you do the swedish social se...
1016,@user2242,20,PROPN,O,@user2046 further you do the swedish social se...


In [4]:
# Changing all @user to be B-PER for now
new_label = combined_df['NER Label'].tolist()
new_pos = combined_df['POS Label'].tolist()

for index, row in combined_df.iterrows():
    
    # retrieving the the information of a particular row
    particular_word = row['Word']
    particular_start_index = row['Word Index']
    original_sentence = row['Tweet']

    # splitting the original sentence
    original_sentence_split = original_sentence.split(' ')
    to_feed_sentence = original_sentence
    to_feed_split = original_sentence_split

    print('-' * 50)
    print('Original Particular word')
    print(particular_word)
    print("Original Sentence Split")     
    print(original_sentence_split)
    print("Original Start Index")
    print(particular_start_index)

    # setting up the initial parameters  
    char_index = 0
    iterate_word_index = 0
    is_hashtag = False
    encode = ''
    encode_len = 0

    # while iterate_word_index < particular_start_index:
    #     particular_word_in_sentence = original_sentence_split[iterate_word_index]
    #     char_index += len(particular_word_in_sentence)
        
    #     # accounting for the space
    #     char_index += 1
    #     iterate_word_index += 1

   

    # if a particular word is a user tag - label it has a person
    if '@user' in particular_word:
        print("CHANGE USER ONLY")
        print("-" * 50)
        new_label[index] = 'B-PER'
        new_pos[index] = 'User: No change'
    else:
        # PRE-PROCESSING THE ENTIRE SENTENCE
        print("FINDING ENTITY OF WORD")

        pre_processed_sentence_split = original_sentence_split
        pre_processed_word_start_index = particular_start_index
        pre_processed_char_start_index = 0
        pre_processed_char_start_index_mushed = 0
        pre_processed_sentence_split_string = original_sentence
        is_hashtag = False

        # if entity is hashtag
        pre_processed_sentence_split_combined = original_sentence_split
        pre_processed_word_start_index_combined = particular_start_index
        pre_processed_char_start_index_combined = 0
        pre_processed_char_start_index_mushed_combined = 0
        pre_processed_sentence_split_string_combined = original_sentence

        part_of_sentence_before_particular_word_combined = []
        part_of_sentence_before_particular_word_string_combined = ''
        pre_processed_char_start_index_combined = 0

        part_of_sentence_before_particular_word_string_mushed_combined = ''
        pre_processed_char_start_index_mushed_combined = 0

        combined_word_length = len(particular_word)
        split_word_length = 0
        

        # checking if a particular word is a hashtag or not
        particular_word_check_for_hashtag = particular_word[0:1]
        if particular_word_check_for_hashtag == '#':
            print("ENTITY IS HASHTAG, WILL RUN FOR COMBINED HASHTAG AS WELL")
            is_hashtag = True
            combined_word_length -= 1

        word_index = 0 
        while word_index in range(len(pre_processed_sentence_split)):
            word_in_sentence = pre_processed_sentence_split[word_index]
            check_for_hashtag = word_in_sentence[0:1]
            to_add_word_index = 1

            # if hashtag, split the sentence
            if check_for_hashtag == '#' or (check_for_hashtag != '#' and word_index == pre_processed_word_start_index):
                if is_hashtag == True:
                    word_in_sentence = word_in_sentence[1:]
                
                print(word_in_sentence)
                word_in_sentence = wordninja.split(word_in_sentence)
                print(word_in_sentence)
                to_add_word_index = len(word_in_sentence)
                to_add_word_string = ' '.join(word_in_sentence)
                to_add_word_string_length = len(to_add_word_string)

                # if check_for_hashtag != '#' and word_index == pre_processed_word_start_index:
                #     print(split_word_length)
                #     split_word_length = len(''.join(word_in_sentence))
                
                # replacing the original phrase with the split
                pre_processed_sentence_split = pre_processed_sentence_split[:(word_index)] + word_in_sentence + pre_processed_sentence_split[(word_index + 1):]

                if (word_index < pre_processed_word_start_index):
                    pre_processed_word_start_index += (to_add_word_index - 1)
             
            word_index += to_add_word_index
        
        

        pre_processed_sentence_split_string = ' '.join(pre_processed_sentence_split)

        # Getting the specific character of the post pre-processed sentence
        part_of_sentence_before_particular_word = pre_processed_sentence_split[:pre_processed_word_start_index]
        part_of_sentence_before_particular_word_string = ' '.join(part_of_sentence_before_particular_word)

        pre_processed_char_start_index = (len(part_of_sentence_before_particular_word_string)) + 1

        # Without getting the string without spaces
        part_of_sentence_before_particular_word_string_mushed = ''.join(part_of_sentence_before_particular_word)
        pre_processed_char_start_index_mushed = (len(part_of_sentence_before_particular_word_string_mushed))
        

        word_index = 0 
        while word_index in range(len(pre_processed_sentence_split_combined)):
            word_in_sentence = pre_processed_sentence_split_combined[word_index]
            check_for_hashtag = word_in_sentence[0:1]
            to_add_word_index = 1

            # if hashtag, split the sentence
            if check_for_hashtag == '#':
                pre_processed_sentence_split_combined[word_index] = word_in_sentence[1:]

                if word_index != pre_processed_word_start_index_combined:
                    word_in_sentence = wordninja.split(word_in_sentence)
                    to_add_word_index = len(word_in_sentence)
                    to_add_word_string = ' '.join(word_in_sentence)
                    to_add_word_string_length = len(to_add_word_string)
                    
                    # replacing the original phrase with the split
                    pre_processed_sentence_split_combined = pre_processed_sentence_split_combined[:(word_index)] + word_in_sentence + pre_processed_sentence_split_combined[(word_index + 1):]

                    if (word_index < pre_processed_word_start_index_combined):
                        pre_processed_word_start_index_combined += (to_add_word_index - 1)
            
            word_index += to_add_word_index

        pre_processed_sentence_split_string_combined = ' '.join(pre_processed_sentence_split_combined)

        # Getting the specific character of the post pre-processed sentence
        part_of_sentence_before_particular_word_combined = pre_processed_sentence_split[:pre_processed_word_start_index_combined]
        part_of_sentence_before_particular_word_string_combined = ' '.join(part_of_sentence_before_particular_word_combined)

        pre_processed_char_start_index_combined = (len(part_of_sentence_before_particular_word_string_combined)) + 1

        # Without getting the string without spaces
        part_of_sentence_before_particular_word_string_mushed_combined = ''.join(part_of_sentence_before_particular_word_combined)
        pre_processed_char_start_index_mushed_combined = (len(part_of_sentence_before_particular_word_string_mushed_combined))
            

        
        # encoding into spacy document to get the NER entity of each word
        sentence_doc = nlp(pre_processed_sentence_split_string) # splitted word phrase
        word_entities = [(e.text, e.ent_iob_, e.ent_type_, e.pos_) for e in sentence_doc]


        sentence_doc_combined = nlp(pre_processed_sentence_split_string_combined)
        word_entities_combined = [(f.text, f.ent_iob_, f.ent_type_, f.pos_) for f in sentence_doc_combined]
        

        # check if the length of the entities that fed in matches with the len of entities that came out
     
        print("FOR NON COMBINED WORDS")

        print("After processing")
        print(pre_processed_sentence_split)
        print("After processing joint sentence")
        print(pre_processed_sentence_split_string)
        print("Pre processed word start index")
        print(pre_processed_word_start_index)
        print("Pre processed char start index")
        print(pre_processed_char_start_index)
        print("Pre processed split string")
        print(pre_processed_sentence_split_string)
        print("Part of sentence before particular word")
        print(part_of_sentence_before_particular_word)
        print("Part of sentence before particular word string")
        print(part_of_sentence_before_particular_word_string)
        print("Part of sentence before particular word string mushed")
        print(part_of_sentence_before_particular_word_string_mushed)
        print("FINAL: Pre Processed char start index mushed")
        print(pre_processed_char_start_index_mushed)


        print("FOR COMBINED WORDS:")

        print("After processing combined")
        print(pre_processed_sentence_split_combined)
        print("After processing joint sentence combined")
        print(pre_processed_sentence_split_string_combined)
        print("Pre processed word start index combined")
        print(pre_processed_word_start_index_combined)
        print("Pre processed char start index combined")
        print(pre_processed_char_start_index_combined)
        print("Pre processed split string combined")
        print(pre_processed_sentence_split_string_combined)
        print("Part of sentence before particular word combined")
        print(part_of_sentence_before_particular_word_combined)
        print("Part of sentence before particular word string combined")
        print(part_of_sentence_before_particular_word_string_combined)
        print("Part of sentence before particular word string mushed combined")
        print(part_of_sentence_before_particular_word_string_mushed_combined)
        print("FINAL: Pre Processed char start index mushed combined")
        print(pre_processed_char_start_index_mushed)
        
        
        print('Word Entities result')
        print(word_entities)

        print("Word Entities COMBINED result")
        print(word_entities_combined)
       
  
        # retrieving the entities to verify:

        # NON COMBINED SENTENCE 
        entities_index_non_combined = 0
        entities_char_non_combined = 0

        word_entities_to_verify_non_combined = []
        print("PRINTING NON COMBINED")
        for entity in word_entities:
            if entities_char_non_combined < pre_processed_char_start_index_mushed:
                print(entity)
                particular_entity_word = entity[0]
                particular_entity_word_length = len(particular_entity_word)
                entities_index_non_combined += 1
                entities_char_non_combined += particular_entity_word_length
            
        
        while entities_char_non_combined < (pre_processed_char_start_index_mushed + combined_word_length):
            entity_to_append_non_combined = word_entities[entities_index_non_combined]
            word_entities_to_verify_non_combined.append(entity_to_append_non_combined)
            
            entities_char_non_combined += len(entity_to_append_non_combined[0])
            entities_index_non_combined += 1
        

        # COMBINED SENTENCE
        entities_index_combined = 0
        entities_char_combined = 0

        word_entities_to_verify_combined = []
        print("PRINTING COMBINED")
        for entity in word_entities_combined:
            if entities_char_combined < pre_processed_char_start_index_mushed_combined:
                print(entity)
                particular_entity_word_combined = entity[0]
                particular_entity_word_length_combined = len(particular_entity_word_combined)
                entities_index_combined += 1
                entities_char_combined += particular_entity_word_length_combined
            
        
        while entities_char_combined < (pre_processed_char_start_index_mushed_combined + combined_word_length):
            entity_to_append_combined = word_entities_combined[entities_index_combined]
            word_entities_to_verify_combined.append(entity_to_append_combined)
            
            entities_char_combined += len(entity_to_append_combined[0])
            entities_index_combined += 1


        print("ENTITIES TO VERIFY NON COMBINED")
        print(word_entities_to_verify_non_combined)

        print("ENTITIES TO VERIFY COMBINED")
        print(word_entities_to_verify_combined)

        # retrieving all the labels and pos tags from each phrase (if any)
        label = []
        pos = []

        
        # otherwise append all of the NER tags from the split word entity (if there are)
        for numents in range(len(word_entities_to_verify_non_combined)):
            if word_entities_to_verify_non_combined[numents][2] != '':
                label.append(word_entities_to_verify_non_combined[numents][2])
            pos.append(word_entities_to_verify_non_combined[numents][3])

        for numents in range(len(word_entities_to_verify_combined)):
            if word_entities_to_verify_combined[numents][2] != '':
                label.append(word_entities_to_verify_combined[numents][2])
            pos.append(word_entities_to_verify_combined[numents][3])
        
        print(label)
        print(pos)
        # Filtering - if the label list is not empty: 
        if label != []:
            # Find the most common label tagging
            label_counter = Counter(label)
            label_most_common = (label_counter.most_common(1))[0][0]
            
            # checking the label itself and then labeling it based on the Tweebank labelling standard
            if label_most_common == 'PERSON':
                new_label[index] = 'B-PER'
            elif label_most_common == 'FAC':
                new_label[index] = 'B-LOC'
            elif label_most_common == 'NORP':
                new_label[index] = 'B-ORG'
            elif label_most_common == 'ORG':
                new_label[index] = 'B-ORG'
            elif label_most_common != 'MONEY' or label_most_common != 'QUANTITY':
                new_label[index] = 'B-MISC'

            pos_counter = Counter(pos)
            pos_most_common = (pos_counter.most_common(1)[0][0])
            new_pos[index] = pos_most_common
            
        else: 
            # If label list is empty, Looking through pos taggings to see if we can classify anything
            # if there is a proper noun, then it can still be classified as miscellaneous
            if 'PROPN' in pos: 
                new_label[index] = 'B-MISC'
                new_pos[index] = 'PROPN'
            elif 'NOUN' in pos:
                new_label[index] = 'B-MISC'
                new_pos[index] = 'NOUN'
            else:
                # if it is not able to find a label, then we find what SPACY thinks the POS tagging is
                pos_counter = Counter(pos)
                pos_most_common = (pos_counter.most_common(1)[0][0])
                new_pos[index] = pos_most_common
    
    
 


--------------------------------------------------
Original Particular word
poutine
Original Sentence Split
['rt', '@user1787', ':', 'i', 'asked', 'siri', 'to', 'remind', 'me', 'about', 'my', 'poutine', 'in', 'the', 'office', 'fridge', 'but', 'she', 'misinterpreted', 'and', 'probably', 'alerted', 'the', 'fbi', '.', 'url222', '…']
Original Start Index
11
FINDING ENTITY OF WORD
poutine
['pout', 'in', 'e']
FOR NON COMBINED WORDS
After processing
['rt', '@user1787', ':', 'i', 'asked', 'siri', 'to', 'remind', 'me', 'about', 'my', 'pout', 'in', 'e', 'in', 'the', 'office', 'fridge', 'but', 'she', 'misinterpreted', 'and', 'probably', 'alerted', 'the', 'fbi', '.', 'url222', '…']
After processing joint sentence
rt @user1787 : i asked siri to remind me about my pout in e in the office fridge but she misinterpreted and probably alerted the fbi . url222 …
Pre processed word start index
11
Pre processed char start index
50
Pre processed split string
rt @user1787 : i asked siri to remind me about my 

In [5]:
new_df = combined_df
new_df['Spacy POS Label'] = new_pos
new_df['Spacy NER Label'] = new_label

new_df = new_df.reindex(columns = ['Word', 'POS Label', 'NER Label', 'Spacy POS Label', 'Spacy NER Label', 'Tweet'])
new_df

,Word,POS Label,NER Label,Spacy POS Label,Spacy NER Label,Tweet
0,poutine,PROPN,O,NOUN,B-MISC,rt @user1787 : i asked siri to remind me about...
1,friday,PROPN,O,PROPN,B-MISC,rt @user912 : you know it 's friday when the p...
2,@user444,PROPN,O,User: No change,B-PER,mikel obi & his girl olga diyachenko spend hol...
3,@user1598,PROPN,O,User: No change,B-PER,""" @user1598 : super junior ’s siwon transforms..."
4,gospel,PROPN,O,PROPN,B-MISC,""" do n't move on from the gospel , move into t..."
...,...,...,...,...,...,...
1013,@user70,PROPN,O,User: No change,B-PER,rt @user2041 : @user70 happy birthday man ! le...
1014,@user660,PROPN,O,User: No change,B-PER,rt @user1157 : happy birthday @user660 ! url1209
1015,@user2046,PROPN,O,User: No change,B-PER,@user2046 further you do the swedish social se...
1016,@user2242,PROPN,O,User: No change,B-PER,@user2046 further you do the swedish social se...


In [6]:
new_df.to_csv('logs/relabelled_data_spacy_pass_entire_sentence.tsv', sep='\t', index=False)